# Github Repo Scraper

In [11]:
#import libraries
import os
import re
import bs4
import csv
import time
import requests
import json, codecs
from pprint import pprint
from github import Github
from bs4 import BeautifulSoup

In [12]:
#write results on a file in json format
def write_json(page_objects,page):
    
    path_name = '../spring'
    if not os.path.exists(path_name):
        os.makedirs(path_name)
    
    file = 'data_word_emb' + str(page) + '.json'
    
    with open(os.path.join(path_name,file),'a') as f:
        json.dump(page_objects,f)

In [13]:
#define functions 
def find_tags(tags,each_issue):
    for counter,temp in enumerate(each.find_all('a')):
        
        temp_ls = temp.contents
            
        if len(temp_ls) == 1:
            tags.append(temp_ls[0])
        
        if len(temp_ls) > 1:
            tag = ' '.join([str(elem) for elem in temp_ls])
            tags.append(tag)
            
    remove_special_ch(tags)

In [14]:
#maybe embody it in the find_desc function to reduce iterations.
def remove_special_ch(text):
    
    for i in range(0,len(text)):
        text[i] = text[i].replace('\n',' ')
        text[i] = text[i].replace('\t',' ')
        text[i] = re.sub(r"<.*?>"," ",text[i])
        #maybe better solution is <[^<>]+> matches any character except < or > 
        #one or more times included inside < and > 

In [15]:
def search_all_page(stack_trace,page):
    temp = page.find_all('div',class_='edit-comment-hide')
    for i in temp:
        temp_2 = i.find('td')
        temp_desc = []
        calculate_desc(temp_2,temp_desc,stack_trace,0)
        if stack_trace != []:
            break

In [16]:
def find_desc(each,base_url,description,stack_trace,is_bug):
    
    next_url     = base_url+each.find('a')['href']
    response     = requests.get(next_url)
    html_content = response.content
    dom          = BeautifulSoup(html_content,'html.parser')
    temp         = dom.find('div',class_ = 'edit-comment-hide')
    temp_2       = temp.find('td')
    
    #find description and the stack trace
    calculate_desc(temp_2,description,stack_trace,1)
    
    #search all the page for a stack trace
    if stack_trace == [] and is_bug == 1:
        search_all_page(stack_trace,dom)
    
    #find the person who closed the issue
    who_closed_it = "none"
    temp_3        = dom.find_all('div',class_='TimelineItem-body') 
    for i in temp_3:
        if i.text.find('closed this')!=-1:
            if i.find('a',class_='author Link--primary text-bold') is None:
                continue
            who_closed_it = i.find('a',class_='author Link--primary text-bold').text
    
    #remove special characters from the description and stack trace
    remove_special_ch(description)
    
    if stack_trace != []:
        remove_special_ch(stack_trace)
    
    return who_closed_it

In [17]:
def calculate_desc(html_text,description,stack_trace,flag):
    
    html_content = html_text.contents
    length = len(html_content)
    
    if length == 1:
        #some lines are empty. No need to write them
        if html_content[0]!= []:
            str_temp = str(html_content[0])
            if str_temp.find('.java:')!=-1 or str_temp.find('java.')!=-1 or str_temp.find('AndroidRuntime:')!=-1:
                stack_trace.append(str(html_content[0]))
            else:
                if flag == 1:
                    if str(html_content[0]).startswith('<code>') != True:
                        description.append(str(html_content[0]))
    else:
        for i in range(length):
            
            if type(html_content[i]) is bs4.element.NavigableString:
                #avoid writting empty lines and html tags <br>
                if len(html_content[i])>4:
                        str_temp = str(html_content[i])
                        if str_temp.find('.java:')!=-1 or str_temp.find('java.')!=-1 or str_temp.find('AndroidRuntime:')!=-1:
                            stack_trace.append(str(html_content[i]))
                        elif flag == 1:
                            description.append(str(html_content[i]))
            elif type(html_content[i] is bs4.element.Tag):
                #call recursively the function till length is 1
                calculate_desc(html_content[i],description,stack_trace,flag)
        

In [18]:
#search all closed issues

#define some variables
total_issues  = 0
total_traces  = 0
total_bugs    = 0
pages_counter = 400
issues        = []


#initialize important url's
base_url = "https://github.com/"
#query_url = f"https://github.com/cgeo/cgeo/issues?page=1&q=is%3Aissue+is%3Aclosed"
#query_url = f"https://github.com/cgeo/cgeo/issues?page=1&q=is%3Aissue+is%3Aopen"
#query_url = f"https://github.com/elastic/elasticsearch/issues?page=1&q=is%3Aissue+is%3Aclosed"
query_url = f"https://github.com/spring-projects/spring-boot/issues?page=401&q=is%3Aissue+is%3Aclosed"

#authorization in order to make more requests.
token = os.getenv("GITHUB_TOKEN")
headers = {'Authorization': f'token {token}'}

response = requests.get(query_url, headers = headers)

In [19]:
#loop through all pages

while True:
    response_code = response.status_code
    
    if response_code != 200:
        raise Exception("Error Occured")
    else:
        html_content = response.content
        dom = BeautifulSoup(html_content,'html.parser')
        
        #find all issues in every page
        all = dom.findAll('div', class_='flex-auto min-width-0 p-2 pr-3 pr-md-2')
        
        page_objects = []
        pages_counter += 1
        
        #real scraping begins
        #search all issues per page
        for each in all:
            
            #find tags and who open the issue
            tags=[]
            find_tags(tags,each)
            
            #flag that is activated if the issue is bug
            is_bug = 0
            #for i in tags:
            #    if i == "Bug":
            #        is_bug = 1
            #        total_bugs = total_bugs+1
            
            #find description, stack trace and who closed
            description   = []
            stack_trace   = []
            who_closed_it = find_desc(each,base_url,description,stack_trace,is_bug)
            
            total_issues = total_issues+1
            if stack_trace !=[]:
                total_traces = total_traces +1
            
            print(total_issues,tags[0])
            
            #write dictionary
            if len(tags)>=1:
                issue_object = {'name':tags[0],'tags':tags[1:-1],'opened_by':tags[len(tags)-1],
                                'description':description,'stack_trace':stack_trace,'closed_by':who_closed_it}
            
            
            page_objects.append(issue_object)
        
        #write issues by page 
        write_json(page_objects,pages_counter)
        
        #combine all issues to a global list
        #issues.extend(page_objects)
        
        #visit the next page if exists
        end = dom.findAll('a',class_='next_page')
        if end == []:
            break
        
        next_url = base_url + end[1]['href']
        response = requests.get(next_url)
        
        ##for debugging
        if total_issues % 25 != 0:
            print("##########################\nTHE PAGE##############\n ",pages_counter,"HAS NOT 25 ISSUES",)
        ##
        if total_issues >= 10000:
            break
        
        # Sleep for 60 seconds
        time.sleep(30)

1 Upgrade to Liquibase 3.6.3
2 Upgrade to Mimepull 1.9.11
3 Upgrade to Infinispan 9.4.6.Final
4 Upgrade to Groovy 2.5.6
5 Upgrade to Tomcat 9.0.16
6 Upgrade to Log4j2 2.11.2
7 Upgrade to Artemis 2.6.4
8 Upgrade to Byte Buddy 1.9.10
9 Upgrade to Mysql 8.0.15
10 Upgrade to Undertow 2.0.17.Final
11 Upgrade to Rxjava2 2.2.6
12 Upgrade to Saaj Impl 1.5.1 and deal with the move to Jakarta API dependencies
13 Upgrade to Couchbase Client 2.7.4
14 Upgrade to Thymeleaf Extras Java8time 3.0.3.RELEASE
15 Upgrade to Neo4j Ogm 3.1.7
16 Upgrade to Hibernate 5.2.18.Final
17 Upgrade to Janino 3.0.12
18 Upgrade to Reactor Bom Bismuth-SR16
19 Upgrade to Netty 4.1.33.Final
20 Upgrade to Spring Security Jwt 1.0.10.RELEASE
21 Upgrade to Spring Retry 1.2.4.RELEASE
22 Upgrade to Tomcat 8.5.38
23 Upgrade to Httpcore 4.4.11
24 Upgrade to Httpclient 4.5.7
25 Upgrade to Undertow 1.4.27.Final
26 Use Testcontainers' CassandraContainer rather than our own
27 Import Testcontainers' bom rather than managing modules in

131 The function parameter deleted in DefaultHashOperations is not clear
132 Ambiguous documentation about ConfigurationProperties
133 Inject dependencies into auto-configuration classes more precisely
134 Update InfoContributorAutoConfiguration to inject InfoContributorProperties into the single bean method that uses it
135 Add support for MockK 
136 WebsocketContainer missing with Tomcat >= 9.0.13
137 Start spring-boot application with classLoader
138 Spring-boot-maven-plugin 2.1.2.RELEASE build error
139 spring-boot-maven-plugin 2.1.2.RELEASE build error
140 WebSocket with with Tomcat / HTTP/2 / TLS
141 Wrong version of text banner logo (2.1.2.RELEASE) show 2.1.1.RELEASE
142 No default constructor found
143 Review "Working with Git branches"
144 Investigate splitting configuration and auto-configuration
145 Java 11 and ForkJoinPool.commonPool() class loading issue
146 failed to find org.cloudera.logredactor
147 spring-boot-loader  classpath 
148 Provide the ability to preset propert

259 Document that   @ConditionalOnClass   should not be used directly on methods
260 LoggingSystemProperties  costs too much time to attain the ApplicationPid
261 Avoid using @ConditionalOnClass(Flux.class) and similar conditions
262 Not able to fetch data in SpringBoot using RedisTemplate class
263 /resources/text.txt does not exist
264 Upgrade to Hibernate ORM 5.4.0
265 How to create custom annotation using @KafkaListener to seek from latest offset always. 
266 use boot 2.1.1 and spring data jpa 
267 Restore reference to nutz third party starter
268 set custom property when the value is on
269 hiddenHttpMethodFilter 
270 Add @ContributeInfo annotation
271 HierarchicalUriComponents.encodeUriComponent() method can not encode Pchar
272 Postman throwing an exception as:- Servlet.service() for servlet [dispatcherServlet] in context with path [] threw exception [Request processing failed; nested exception is org.json.JSONException: JSONObject["name"] not found.] with root cause  org.json.J

382 Upgrade to Spring Security 5.0.11
383 Upgrade to Spring Security 4.2.11
384 Upgrade to Spring Kafka 2.2.3
385 Upgrade to Spring Integration 5.1.2
386 Upgrade to Spring Integration 5.0.11
387 Upgrade to Spring Integration 4.3.19
388 Upgrade to Spring Data Lovelace SR4
389 Upgrade to Spring Data Kay SR13
390 Upgrade to Spring Data Ingalls SR18
391 Upgrade to Spring AMQP 2.1.3
392 Upgrade to Spring AMQP 2.0.11
393 metrics timer
394 Building project from the source
395 Provide configuration property to set name for RabbitMQ connection factory
396 Upgrade to Flyway 5.2.4
397 YAML not parsed correctly when using @PropertySource
398 Can no longer use ContextConfiguration initializer in 2.1.0 to assign random server port
399 Support space-delimited oauth2 scope
400 Default values of var-based properties are not documented properly
401 ServerResponse.status is invalid in AOP
402  No qualifying bean of type [javax.sql.DataSource]
403 Change "Colophon" to "Legal"
404 Exclude jcl-over-slf4j an

510 Upgrade to Spring Security 5.1.2.RELEASE
511 Upgrade to Spring Kafka 2.2.2.RELEASE
512 Upgrade to Spring Integration 5.1.1.RELEASE
513 Upgrade to Spring AMQP 2.1.2.RELEASE
514 Upgrade to Maven Shade Plugin 3.2.1
515 Upgrade to Selenium Htmlunit 2.33.3
516 Upgrade to Lombok 1.18.4
517 Upgrade to Mockito 2.23.4
518 Upgrade to Jooq 3.11.7
519 Upgrade to Infinispan 9.4.2.Final
520 Upgrade to Elasticsearch 6.4.3
521 Upgrade to Ehcache3 3.6.2
522 Upgrade to Groovy 2.5.4
523 Upgrade to Tomcat 9.0.13
524 Upgrade to Kafka 2.0.1
525 Upgrade to Byte Buddy 1.9.4
526 Upgrade to Undertow 2.0.15.Final
527 Upgrade to Couchbase Client 2.7.1
528 Upgrade to Spring AMQP 2.0.10.RELEASE
529 Upgrade to Spring Security 4.2.10
530 Upgrade to Spring Session Apple-SR7
531 Upgrade to Spring Security 5.0.10
532 Upgrade to Spring Kafka 2.0.7.RELEASE
533 Upgrade to Spring Integration 5.0.10
534 Upgrade to Spring Cloud Connectors 2.0.4.RELEASE
535 Upgrade to Jetty El 8.5.33.1
536 Upgrade to Elasticsearch 5.6.13
5

...

9363 Use random port for Cassandra sample tests
9364 Using JSON instead of Properties
9365 LoggingSystem shutdown on application context stop (logback)
9366 Caused by: java.lang.NoClassDefFoundError: org/slf4j/LoggerFactory
9367 spring-boot spring-security thymeleaf, authentication returns null on error pages
9368 Support for adding multiple JNDI datasources (or JNDI resources in general)
9369 Poor SseEmitter handling of timeouts
9370 Upgrade to Spring Framework 4.2.2.RELEASE
9371 Upgrade to Spring Framework 4.1.8.RELEASE
9372 Unify X-Forwarded for server properties
9373 Improvements to MongoProperties for auto-configuration
9374 Means to override managed Spring Data release train
9375 Prevent Undertow from exposing classpath files
9376 Parent ApplicationContexts are not respected
9377 spring.profiles.include does not seem to be picked up in JBoss 6.3 system properties
9378 DataSource exception message seems misleading
9379 Flyway initializer - higher or customizable priority
9380 spri

9499 Improve "Use a random unassigned HTTP port" section
9500 Spring-starter-mustache: Clarify exception message for checkTemplateLocationExists
9501 ClassNotFoundException when setting context class in SpringApplicationBuilder without spring-web
9502 Spring Boot CLI's   run   command should identify and run Maven & Gradle projects
9503 Add meta-data support for write-only properties on classes annotated with @ConfigurationProperties
9504 "Console output" and --debug flag documentation
9505 The startup script should support system accounts with /sbin/nologin
9506 Allow properties configured by dev tools to be overridden
9507 java.lang.ClassNotFoundException: org.springframework.dao.DataAccessException
9508 Application properties outside of jar should take precedence over profile-specific variants inside the jar
9509 Upgrade to H2 1.4.188
9510 @PropertySource should be able to overwrite the default application.properties
9511 CLI causes error with description option having white spaces


9619 PropertiesLauncher: property "loader.path" and jar with embeded zip file
9620 Typo in documentation
9621 Is it possible to use Spring MVC with Jersey 2.x?
9622 ErrorPage documentation is misleading
9623 Spring boot devtools and orika mapper issue
9624 /hal and /links should not be separate endpoints
9625 Reference docs should contain a section on how to override dependency properties to upgrade to newer version
9626 Offer configuration options to enable OPTIONS and TRACE
9627 spring boot's jetty8 sample rest error!
9628 Links from restdocs won't work if endpoints have been remapped
9629 Make actuator docs work offline
9630 Actuator docs title links don't work
9631 Spring Boot startup failure when running executable war file with libraries containing TypeConverter services file and classes annotated @Converter
9632 Allow setting a custom ThreadPool instance when instantiating Server in JettyEmbeddedServletContainerFactory
9633 Consider reverting removal of dead code from Configurat

9750 Spring boot security properties set doen't work
9751 Auto-configure AllEncompassingFormHttpMessageConverter's part converters
9752 Revert Pivotal education changes in gh-pages
9753 Provide an example of .conf file
9754 Make init script LSB compliant
9755 Wrong PID_FOLDER default value in documentation
9756 Expose frequent listener settings for the default   JmsListenerFactory 
9757 FATAL log level is mapped to ERROR even for log systems that supports it.
9758 Spring boot security properties set doen't work
9759 Spring boot startup is resolving the same property 34 times
9760 Allow Tomcat connector's accept count to be configured via the environment
9761 Update a sample to showcase the use of configurationPropertiesValidator
9762 How to build custom PropertySource for binding to @ConfigurationProperties
9763 Validation of @ConfigurationProperties does not happen for nested properties
9764 Allow SSL on main embedded servlet container instance (and not on manager instance)
9765 When 

9878 Rename ResourceProperties.chain.html5AppCache
9879 Consider if we should always register ResourceUrlEncodingFilter
9880 Memory consumption increases beyond -Xmx option
9881 Property to enable SPDY / HTTP2.0 in Undertow server
9882 Fix Windows build
9883 Write an AMQP section in the documentation
9884 Support bootRun-style inclusion of src/main/resources on classpath when running in-IDE
9885 Overhaul actuator endpoint MVC configuration 
9886 Devtools does not work with Jetty
9887 Build fails when use mvn clean install
9888 Support conditional configuration in logback based on active Spring profiles
9889 MappingJackson2HttpMessageConverter with Jackson2HalModule registered ObjectMapper  was not added to the message converts
9890  javax.management.MBeanServer
9891 gradle spring-boot plugin 1.3.0.M1 does not work with gradle 2.5-rc-1
9892 Devtools: Enable automatic logins at dev time
9893 Allow expectations to be set on OutputCapture
9894 Fail the build if the additional meta-data for

In [20]:
#print some statistics

print(total_traces)
print(total_issues)

1585
10000
